In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

In [12]:
import nltk

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [16]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [17]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [18]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [19]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [20]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\user\AppData\Local\Temp\ipykernel_7332\2836083368.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [27]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

In [28]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1e-6,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [29]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [30]:
#fitting the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2062 - accuracy: 0.1702
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.2775 - accuracy: 0.0426
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.2234 - accuracy: 0.0851
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1245 - accuracy: 0.1915
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1496 - accuracy: 0.1702
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1576 - accuracy: 0.1489
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1841 - accuracy: 0.1489
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1112 - accuracy: 0.2128
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1001 - accuracy: 0.2766
Epoch 10/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0813 - accuracy: 0.1702
Epoch 11/

10/10 [==============================] - 0s 1ms/step - loss: 1.2336 - accuracy: 0.6809
Epoch 83/200
10/10 [==============================] - 0s 786us/step - loss: 1.0939 - accuracy: 0.7872
Epoch 84/200
10/10 [==============================] - 0s 804us/step - loss: 1.2296 - accuracy: 0.6809
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 1.1534 - accuracy: 0.7234
Epoch 86/200
10/10 [==============================] - 0s 1ms/step - loss: 1.0803 - accuracy: 0.6596
Epoch 87/200
10/10 [==============================] - 0s 771us/step - loss: 1.1443 - accuracy: 0.7021
Epoch 88/200
10/10 [==============================] - 0s 796us/step - loss: 1.0762 - accuracy: 0.8085
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 1.1509 - accuracy: 0.6596
Epoch 90/200
10/10 [==============================] - 0s 1ms/step - loss: 0.9150 - accuracy: 0.7872
Epoch 91/200
10/10 [==============================] - 0s 780us/step - loss: 1.0927 - accuracy: 0.7234
Epo

10/10 [==============================] - 0s 1ms/step - loss: 0.4653 - accuracy: 0.8936
Epoch 163/200
10/10 [==============================] - 0s 1ms/step - loss: 0.4813 - accuracy: 0.8511
Epoch 164/200
10/10 [==============================] - 0s 1ms/step - loss: 0.4361 - accuracy: 0.8936
Epoch 165/200
10/10 [==============================] - 0s 775us/step - loss: 0.5100 - accuracy: 0.8936
Epoch 166/200
10/10 [==============================] - 0s 767us/step - loss: 0.4582 - accuracy: 0.8936
Epoch 167/200
10/10 [==============================] - 0s 990us/step - loss: 0.5576 - accuracy: 0.8723
Epoch 168/200
10/10 [==============================] - 0s 997us/step - loss: 0.5678 - accuracy: 0.8511
Epoch 169/200
10/10 [==============================] - 0s 886us/step - loss: 0.4308 - accuracy: 0.8936
Epoch 170/200
10/10 [==============================] - 0s 762us/step - loss: 0.4145 - accuracy: 0.8936
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.4013 - accuracy:

In [31]:
# Saving the model
model.save('chatbot_model.h5', hist)
print("model created")

model created
